In [1]:
!python3 --version

Python 3.8.0


In [2]:
!nvidia-smi

Mon Mar 27 17:03:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:C1:00.0 Off |                    0 |
| N/A   23C    P0    34W / 250W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!python -m pip install ekphrasis
!python -m pip install transformers
!python -m pip install imblearn
!python -m pip install pandas
!python -m pip install sklearn
!python -m pip install numpy
!python -m pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install textattack
!python -m pip install transformers
!python -m pip install ipywidgets==7.4.2
!python -m pip install mlflow
!jupyter nbextension enable --py widgetsnbextension

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Defaulting to user installation because normal site-packages is not writeable


  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)


  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.2.0
    Uninstalling importlib-metadata-5.2.0:
      Successfully uninstalled importlib-metadata-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 6.0.0 requires importlib-metadata>=4.8; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
import os
import time
import numpy as np
import pandas as pd
import sklearn.metrics
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import re
from _datetime import datetime as dt
import random
import torch
from tqdm import tqdm
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    get_linear_schedule_with_warmup
)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, TensorDataset
import datetime
from sklearn.metrics import classification_report, f1_score
from imblearn.under_sampling import RandomUnderSampler
import json
import mlflow

2023-03-27 17:04:17.376621: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 17:04:18.243957: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory
2023-03-27 17:04:18.244199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory
2023-03-27 17:04:18.244209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia 

In [6]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

img_1 = re.compile('!\[(.*)\]\(.*\)')
link_1 = re.compile('\[(.*)\]\(.*\)')
link_2 = re.compile('\[(.*)\]: [^\s]+')
code_1 = re.compile('(:?`[^`]+`|```[^`]*```)')

def preprocess(row):
  doc = ""
  doc += str(row.issue_title)
  doc += " "
  doc += str(row.issue_body)
  
  return clean_text(doc)

def clean_text(text):
  cleaned = re.sub(img_1, r'\1 <img>', text)
  cleaned = re.sub(link_1, r'\1 <url>', cleaned)
  cleaned = re.sub(link_2, r'\1 <url>', cleaned)
  cleaned = re.sub(code_1, '<code>', cleaned)
  ekph_cleaned = " ".join(text_processor.pre_process_doc(cleaned))
  return ekph_cleaned


/home/novielli/.local/lib/python3.8/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/home/novielli/.local/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [7]:
def preprocess_rows(df, label_encoder):
    df = df.fillna({
                        'issue_title': '',
                        'issue_body':''                  
                   })
    df['text'] = df['issue_title'] + df['issue_body']
    df['label'] = label_encoder.transform(df['issue_label'])
    df = df.filter(['text', 'label'])
    df['text'] = [clean_text(text) for text in tqdm(df['text'])]
    return df

In [8]:
def classification_report_to_dataframe(str_representation_of_report):
    split_string = [x.split(' ') for x in str_representation_of_report.split('\n')]
    column_names = ['']+[x for x in split_string[0] if x!='']
    values = []
    for table_row in split_string[1:-1]:
        table_row = [value for value in table_row if value!='']
        if table_row!=[]:
            values.append(table_row)
    for i in values:
        for j in range(len(i)):
            if i[1] == 'avg':
                i[0:2] = [' '.join(i[0:2])]
            if len(i) == 3:
                i.insert(1,np.nan)
                i.insert(2, np.nan)
            else:
                pass
    report_to_df = pd.DataFrame(data=values, columns=column_names)
    return report_to_df

In [ ]:
!curl -LJO "https://zenodo.org/record/7777793/files/github-labels-top3-803k-train-filtered-urls.csv"
!curl -LJO "https://zenodo.org/record/7777793/files/github-labels-top3-803k-test-filtered-urls.csv"

In [ ]:
if not os.path.isfile("github-labels-top3-803k-train.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-train.tar.gz" | tar -xz
if not os.path.isfile("github-labels-top3-803k-test.csv"):
  !curl "https://tickettagger.blob.core.windows.net/datasets/github-labels-top3-803k-test.tar.gz" | tar -xz

In [ ]:
trainset = pd.read_csv("github-labels-top3-803k-train.csv")
testset = pd.read_csv("github-labels-top3-803k-test.csv")

In [ ]:
trainset_filtered_urls = pd.read_csv("github-labels-top3-803k-train-filtered-urls.csv")
testset_filtered_urls = pd.read_csv("github-labels-top3-803k-test-filtered-urls.csv")

In [ ]:
trainset.drop_duplicates(subset=['issue_url'], inplace=True)
testset.drop_duplicates(subset=['issue_url'], inplace=True)
trainset_filtered_urls.drop_duplicates(inplace=True)
testset_filtered_urls.drop_duplicates(inplace=True)

In [ ]:
train_df = pd.merge(trainset_filtered_urls, trainset, on=['issue_url'], how='inner')
test_df = pd.merge(trainset_filtered_urls, testset, on=['issue_url'], how='inner')

In [11]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [12]:
lenc = LabelEncoder()
lenc.fit(trainset["issue_label"])

LabelEncoder()

In [14]:
with open('dict.json') as f:
    memo_dict = json.load(f)

In [15]:
#AGE_FILTER = 365 * 4
AGE_FILTER = (365 * 4) + 1
STAR_FILTER = 1500

In [16]:
stars = []
age = []

temp = trainset.copy()
for ix, x in tqdm(temp.iterrows()):
    stars.append(memo_dict[x['repository_url']][0])
    age.append(memo_dict[x['repository_url']][1])
    
temp['stars'] = stars
temp['age'] = [int(x) for x in age]

581135it [00:22, 25793.27it/s]


In [17]:
#filtered_df_train = temp.loc[(temp['age']>= AGE_FILTER/4) & (temp['age']<= AGE_FILTER)]
filtered_df_train = temp.loc[(temp['age']>= AGE_FILTER) & (temp['stars']>= STAR_FILTER)]

In [ ]:
filtered_df_train


In [19]:
starss = []
agee = []

tempp = testset.copy()
for ix, x in tqdm(tempp.iterrows()):
    starss.append(memo_dict[x['repository_url']][0])
    agee.append(memo_dict[x['repository_url']][1])

tempp['stars'] = starss
tempp['age'] = agee

65951it [00:02, 25438.41it/s]


In [20]:
#filtered_df_test = tempp.loc[(tempp['age']>= AGE_FILTER/4) & (tempp['age']<= AGE_FILTER)]
filtered_df_test = tempp.loc[(tempp['age']>= AGE_FILTER) & (temp['stars']>= STAR_FILTER)]
#filtered_df_test = tempp.loc[(temp['age']>= AGE_FILTER) & (temp['age']<= AGE_FILTER*4 ]

In [ ]:
filtered_df_test

In [ ]:
filtered_df_train

In [24]:
label_encoder = LabelEncoder()
label_encoder.fit(trainset['issue_label'])

LabelEncoder()

In [25]:
from datasets import Dataset

train_set = preprocess_rows(filtered_df_train, label_encoder)
test_set = preprocess_rows(filtered_df_test, label_encoder)

100%|██████████| 3858/3858 [00:07<00:00, 547.53it/s]


In [26]:
train_size = filtered_df_train.groupby("issue_label").size()
train_size = [train_size['bug'], train_size['enhancement'], train_size['question']]
index = train_size.index(min(train_size))

In [27]:
from imblearn.under_sampling import RandomUnderSampler

under_train_dict = {0:train_size[index] ,1:train_size[index],2:train_size[index]}

train_set, _ = RandomUnderSampler(sampling_strategy=under_train_dict, random_state=42).fit_resample(train_set, list(train_set["label"]))

In [28]:
train_set = Dataset.from_pandas(train_set)
test_set = Dataset.from_pandas(test_set)

In [29]:
BERT_MODEL = 'roberta-base'
NUM_LABELS = 3


In [30]:
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL, model_max_length=128)

In [31]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [32]:
tokenized_dataset_train = train_set.map(tokenize_function, batched=True)
tokenized_dataset_test = test_set.map(tokenize_function, batched=True)

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [33]:
LEARNING_RATE = 2e-5
WEIGHT_DECAY= 1e-8
NUM_TRAIN_EPOCHS = 4
PER_DEVICE_TRAIN_BATCH_SIZE = 32

In [34]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=NUM_LABELS, 
                                                           max_length=128)



training_args = TrainingArguments(output_dir="test_trainer",
                                 learning_rate=LEARNING_RATE,
                                 weight_decay=WEIGHT_DECAY,
                                 num_train_epochs=NUM_TRAIN_EPOCHS,
                                 per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                                 seed = seed_val)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [35]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
)

In [37]:
trainer.train()

Step,Training Loss
500,0.577400
1000,0.556300
1500,0.525600
2000,0.479500
2500,0.478600
3000,0.433800
3500,0.404100
4000,0.394000
4500,0.364300
5000,0.349300


TrainOutput(global_step=5404, training_loss=0.4475018115153055, metrics={'train_runtime': 527.1048, 'train_samples_per_second': 327.874, 'train_steps_per_second': 10.252, 'total_flos': 1.1368078326355968e+16, 'train_loss': 0.4475018115153055, 'epoch': 4.0})

In [38]:
output = trainer.predict(tokenized_dataset_test)

In [39]:
preds = np.argmax(output.predictions, axis=-1)

In [40]:
print(classification_report(output.label_ids, preds, digits=4))

              precision    recall  f1-score   support

           0     0.9032    0.7751    0.8342      2094
           1     0.7763    0.8290    0.8018      1164
           2     0.5587    0.7617    0.6446       600

    accuracy                         0.7893      3858
   macro avg     0.7461    0.7886    0.7602      3858
weighted avg     0.8113    0.7893    0.7950      3858



In [41]:
cf = classification_report(output.label_ids, preds, digits=4, output_dict=True)

In [42]:
cf

{'0': {'precision': 0.9031719532554258,
  'recall': 0.7750716332378224,
  'f1-score': 0.8342328450269854,
  'support': 2094},
 '1': {'precision': 0.7763475462590507,
  'recall': 0.8290378006872853,
  'f1-score': 0.8018280016618197,
  'support': 1164},
 '2': {'precision': 0.558679706601467,
  'recall': 0.7616666666666667,
  'f1-score': 0.6445698166431595,
  'support': 600},
 'accuracy': 0.7892690513219285,
 'macro avg': {'precision': 0.7460664020386479,
  'recall': 0.7885920335305915,
  'f1-score': 0.7602102211106548,
  'support': 3858},
 'weighted avg': {'precision': 0.8113318916338197,
  'recall': 0.7892690513219285,
  'f1-score': 0.7949593730966203,
  'support': 3858}}

In [43]:
size_train = filtered_df_train.groupby("issue_label").size()
size_test = filtered_df_test.groupby("issue_label").size()

In [44]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(output.label_ids, preds)
cm

array([[1623,  216,  255],
       [  93,  965,  106],
       [  81,   62,  457]])

In [45]:
mlflow.end_run()
experiment_name = "Undersampling"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
    experiment_id=current_experiment['experiment_id']

run_name = str(AGE_FILTER) + "_" + str(STAR_FILTER) + "_undersampling"

with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):
    mlflow.log_param("model_name", BERT_MODEL)
    mlflow.log_param("lr", LEARNING_RATE)
    mlflow.log_param("weight_decay", WEIGHT_DECAY)
    mlflow.log_param("num_train_epochs", NUM_TRAIN_EPOCHS)
    mlflow.log_param("per_device_train_batch_size", PER_DEVICE_TRAIN_BATCH_SIZE)
    mlflow.log_param("age_filter", AGE_FILTER)
    mlflow.log_param("train_bug", under_train_dict[0])
    mlflow.log_param("train_enhancement", under_train_dict[1])
    mlflow.log_param("train_question", under_train_dict[2])
    mlflow.log_param("test_bug", size_test['bug'])
    mlflow.log_param("test_enhancement", size_test['enhancement'])
    mlflow.log_param("test_question", size_test['question'])
    mlflow.log_metric("no_issues", len(train_set))
    mlflow.log_metric("bug.prec", cf['0']['precision'])
    mlflow.log_metric("bug.rec", cf['0']['recall'])
    mlflow.log_metric("bug.f1", cf['0']['f1-score'])
    mlflow.log_metric("bug.supp", cf['0']['support'])
    mlflow.log_metric("enhancement.prec", cf['1']['precision'])
    mlflow.log_metric("enhancement.rec", cf['1']['recall'])
    mlflow.log_metric("enhancement.f1", cf['1']['f1-score'])
    mlflow.log_metric("enhancement.supp", cf['1']['support'])
    mlflow.log_metric("question.prec", cf['2']['precision'])
    mlflow.log_metric("question.rec", cf['2']['recall'])
    mlflow.log_metric("question.f1", cf['2']['f1-score'])
    mlflow.log_metric("question.supp", cf['2']['support'])
    mlflow.log_metric("f1-micro", cf['accuracy'])
    mlflow.log_metric("prec-macro", cf['macro avg']['precision'])
    mlflow.log_metric("rec-macro", cf['macro avg']['recall'])
    mlflow.log_metric("f1-macro", cf['macro avg']['f1-score'])
    mlflow.log_metric("supp", cf['macro avg']['support'])
    mlflow.log_metric("cm_0_0", cm[0][0])
    mlflow.log_metric("cm_0_1", cm[0][1])
    mlflow.log_metric("cm_0_2", cm[0][2])
    mlflow.log_metric("cm_1_0", cm[1][0])
    mlflow.log_metric("cm_1_1", cm[1][1])
    mlflow.log_metric("cm_1_2", cm[1][0])
    mlflow.log_metric("cm_2_0", cm[2][0])
    mlflow.log_metric("cm_2_1", cm[2][1])
    mlflow.log_metric("cm_2_2", cm[2][2])
mlflow.end_run()

In [46]:
import json

path_to_log = 'predictions'
path_to_file = f'{AGE_FILTER} FILTER.json'

os.makedirs(path_to_log, exist_ok=True)

preds = [int(x) for x in preds]

with open(os.path.join(path_to_log, path_to_file), 'w') as f:
    json.dump(preds, f)



In [47]:
list(preds)


[0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 1,


In [48]:
preds

[0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 2,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
